In [1]:
# Step 1: Define the libraries that you will need
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Step 2: Define the paths
dataset_path = 'path_to_your_dataset_directory'

# Load the labels
labels_df = pd.read_csv(os.path.join(dataset_path, 'labels.csv'))

# Load images and preprocess
def load_images_and_labels(labels_df, dataset_path, img_size=(224, 224)):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        img_path = os.path.join(dataset_path, row['image'] + '.jpeg')
        image = cv2.imread(img_path)
        image = cv2.resize(image, img_size)
        images.append(image)
        labels.append(row['level'])
    return np.array(images), np.array(labels)

images, labels = load_images_and_labels(labels_df, dataset_path)

In [ ]:
# Step 3: Preprocess Data
# Split the data into training and testing sets and perform data augmentation:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

datagen.fit(X_train)


In [ ]:
# Step 4: Build the CNN Model
# Create a CNN model using Keras:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 classes for the levels
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Step 5: Train the Model
# Train the CNN model:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))

In [ ]:
# Step 6: Save the Model
# Save the trained model:
model.save('diabetic_retinopathy_model.h5')

In [ ]:
# Step 7: Create a Flask API
# Create a Flask API to serve the model:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2

app = Flask(__name__)

# Load the model
model = load_model('diabetic_retinopathy_model.h5')

def prepare_image(image, target):
    image = cv2.resize(image, target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        image = prepare_image(image, target=(224, 224))

        preds = model.predict(image)
        result = np.argmax(preds, axis=1)[0]
        
        return jsonify({'prediction': int(result)})

if __name__ == '__main__':
    app.run(debug=True)
